In [3]:
import polars as pl
import pandas as pd
from itertools import product
import multiprocessing

In [4]:
#Get dataframe of all tickers
constituents = pl.read_csv("../Data/SP500/Constituents.csv", infer_schema_length=False)

#Create list of individual stocks from names from dataframe
stockComponents=[]
for k in range(len(constituents)):
    stockComponents.append(constituents[k,0])
    #stockComponents.append(constituents[k,0]+'.L')

In [5]:
#Load prices and timestamps from CSVs
openPrices=[]
closePrices=[]
timestamps=[]
for k in range(len(stockComponents)):
    currentDf=pl.read_csv("../Data/SP500/dailyHist2000/"+stockComponents[k]+".csv", infer_schema_length=None)
    openPrices.append(currentDf['Open'])
    closePrices.append(currentDf['Close'])
    timestamps.append(currentDf['timestamp'])

In [25]:
def algo(paramsList):
    change=paramsList[0]
    takeProfit=paramsList[1]
    stopLoss=paramsList[2]
    timeout=paramsList[3]
    
    equalTradePercent=0
    comboTradePercent=1

    total=0
    totalTime=0

    for k in range(len(openPrices)):

        start=10
        end=int(len(openPrices[k])-((timeout/86400)+1))

        # equalTradePercent=0
        # comboTradePercent=1

        for j in range(start, end):
            if openPrices[k][j]>openPrices[k][j-1]*change:
                total+=1
                buyPrice = openPrices[k][j]

                startTime=timestamps[k][j]
                time=0
                ratio=1

                datapoint=j+1
                while time < timeout and stopLoss<ratio<takeProfit:
                    time = int(timestamps[k][datapoint])-startTime
                    sellPrice=closePrices[k][datapoint]
                    ratio = sellPrice/buyPrice
                    datapoint+=1
                
                equalTradePercent+=ratio
                comboTradePercent*=ratio
                totalTime+=time
        
    try:
        equalTradePercent=equalTradePercent/total
        comboTradePercent=comboTradePercent**(1/total)
    except:
        pass

    averageTime=((totalTime/86400)/total)+1
    equalTradePerDay = (equalTradePercent**(1/averageTime))
    comboTradePerDay = (comboTradePercent**(1/averageTime))
    #return equalTradePerDay, comboTradePerDay
    print(equalTradePercent, comboTradePercent)
    print(equalTradePerDay, comboTradePerDay)

In [8]:
#change
minimum=1.00
maximum=1.10
difference=0.01
changeList=[]
j=minimum
while j<maximum+difference:
    changeList.append(j)
    j+=difference

#takeProfit
minimum=1.01
maximum=1.02
#maximum=1.10
difference=0.01
takeProfitList=[]
j=minimum
while j<maximum+difference:
    takeProfitList.append(j)
    j+=difference

#stopLoss
#minimum=0.90
minimum=0.98
maximum=0.99
difference=0.01
stopLossList=[]
j=minimum
while j<maximum+difference:
    stopLossList.append(j)
    j+=difference

#timeout
minimum=3*86400
maximum=6*86400
difference=1*86400
timeoutList=[]
j=minimum
while j<maximum+difference:
    timeoutList.append(j)
    j+=difference

In [26]:
fullCombinations=list(product(changeList, takeProfitList, stopLossList, timeoutList))
list(fullCombinations[0])
algo(fullCombinations[0])

1.001120563434722 inf
1.0003228235626176 inf


In [23]:
for k in range(len(fullCombinations)):
    algo(fullCombinations[k])

1.0003228235626176 inf


KeyboardInterrupt: 

In [18]:
with multiprocessing.Pool(40) as pool:
    pool.map(
        algo,
        fullCombinations
    )